In [1]:
import xml.etree.ElementTree as ET
# annotation한 csv를 읽음.
def read_annotation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = ['helmet', 'non_helmet', 'vest', 'non_vest']

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [2]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        # jpg와 png 인 이미지 파일들
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_annotation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))
            # 수집한 이미지에서 xml로 저장해둔 데이터 읽음.
    images = np.array(images)

    return images, annotations

In [3]:
'''
# 동일한 디렉토리 내에 image set과 annotation set을 준비한 후 프로그램 실행
'''

'\n이미지와 라벨링한 xml을 한 폴더에 넣고 실행.\n'

In [11]:
import imgaug as ia
from imgaug import augmenters as iaa
#imgaug 라이브러리 설치
from files import *
from pascal_voc_writer import Writer
#pascal voc 파일 형식으로 이미지 주석 xml 파일을 만드는데 사용

ia.seed(1) #난수 생성

dir = 'c:/upload/'
images, annotations = read_train_dataset(dir)
'''
read_train_dataset()
image 파일들과 각 이미지의 annotation xml 파일들이 함께 있는 디렉토리의 경로를 인자로 전달했을 때,
4차원(N:이미지 수, W:이미지 너비, H:이미지 높이, D:RGB) nparray로 변환된 이미지들과 
3개의 항(bounding-box 리스트, xml 파일명, 관련 이미지 파일명)을 갖는 Annotation 투플의 리스트를 반환한다.
'''

for idx in range(len(images)):
    image = images[idx]
    boxes = annotations[idx][0]

    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))
    # 처음 바운딩 박스 정보.
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape =image.shape)

    seq = iaa.Sequential([
        iaa.Multiply((0.7, 1.5)),
        #0.7~1.5 배의 밝기 값 변환
        iaa.Affine(
            translate_px={"x": (-30,30), "y": (-30,30)}, #-30~30 픽셀의 평행이동
            scale=(0.5, 1), #0.5~1배의 scale 변환
            rotate=(-15,15) #-15~15도 회전.
        ),
        iaa.Fliplr(0.5) #0.5의 확률로 좌우 반전의 옵션이 랜덤하게 들어감
    ])

    seq_modify = seq.to_deterministic()
    # 증대시킨 후의 이미지를 저장.
    image_aug = seq_modify.augment_images([image])[0]
    bbs_aug = seq_modify.augment_bounding_boxes([bbs])[0]

    new_image_file = dir + 'after_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))
        # 증대시킨, 라벨린 된 x1, x2, y1, y2 대로 맞춰주기.
    voc_writer.save(dir + 'after_' + annotations[idx][1])
    # after_ 를 붙여서 저장.
    
#가지고 있는 이미지와 해당 annotation 에 적용        

C:\Users\user\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
'''
0.5~1.5배의 밝기 값 변환, -30~30 픽셀의 평행이동, 
0.5~1배의 scale 변환, -15~15도 회전, 0.5의 확률로 좌우 반전의 옵션이 랜덤하게 들어가면서 이미지를 변화시킨다
'''